# Preprocessing for Nitrogen Loading Model (Nload)

## Objectives

Within subwatersheds of Tidmarsh, reclassify Massachusetts 2016 land cover layer to include cranberry agriculture into 13 categories for use in Nload. Repeat this for baseline conditions, and several scenarios.

Future scenarios could include increasing population and impervious area by displacing natural areas. Growth of cranberry agriculture in uplands with modern cranbery hybrids. Retirement and/or restoration of older and flowthrough bogs. Replacement of older flowthrough bogs with new hybdrids. Upgrades to wastewater treatment facilities. 


## References 
Kennedy, C. D., & Hoekstra, B. R. (2021). Measuring and modeling nitrogen export from cranberry farms. Ecosphere, 12(12), 1–15. https://doi.org/10.1002/ecs2.3686

Valiela, I., Geist, M., McClelland, J., & Tomasky, G. (2000). Nitrogen loading from watersheds to estuaries: Verification of the Waquoit Bay Nitrogen Loading Model. Biogeochemistry, 49(3), 277–293. https://doi.org/10.1023/A:1006345024374

Valiela, I., Collins, G., Kremer, J., Lajtha, K., Geist, M., Seely, B., … Sham, C. H. (1997). Nitrogen loading from coastal watersheds to receiving estuaries: New method and application. Ecological Applications, 7(2), 358–380. https://doi.org/10.1890/1051-0761(1997)007[0358:NLFCWT]2.0.CO;2

## Set up environment

In [149]:
# Load dependancies ***************************************
import arcpy as ap
import os, glob, re, sys
import pandas as pd
import numpy as np
#import scipy.stats

# Set up arcpy environment ******************************
# input data directory
idr = "C:\\Workspace\\Geodata\\Massachusetts\\"
# working directory this should be where arcgis project is located
wdr = "C:\\Workspace\\Geodata\\Nload\\"
# output directory
odr = os.path.join(wdr,'outputs')
# arcpy settings
ap.env.workspace = wdr # set arcpy environment working directory
aprx = arcpy.mp.ArcGISProject("Current") # current project
ap.CheckOutExtension("Spatial") # check out spatial extension
ap.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 UTM Zone 19N") # define cordinate system
# Allow overwriting of output
arcpy.env.overwriteOutput = True
#ap.SetProgressor("step", "processing...", step_value = 5)

Tip: see this [page](https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/setprogressor.htm) for tips on arcpy settings. 

Tip: Use `os.path.join` to safely concatenate file paths. To do this enter folder names in quotes separated by commas.

Tip: Test in the geoprocessing tab first, then, once working, can copy python code from the geoprocessing tab when you click on the arrow next to `run`!

In [150]:
# check that directories exist
if os.path.exists(wdr) is False:
    sys.exit(wdr+" does not exist")
if os.path.exists(idr) is False:
    sys.exit(idr+" does not exist")
if os.path.exists(odr):
    print(odr+" already exists")
else:
    os.mkdir(odr)

C:\Workspace\Geodata\Nload\outputs already exists


Tip: Use `arcpy.FeatureSet` to add an input feature as an object to the current environment. Alternatively, `arcpy.management.MakeFeatureLayer` can be used to add input feature as a layer to the current project. The added layer should appear on the map. `arcpy.Copyfeatures_management` works in a similar way to `MakeFeatureLayer`. 

## Load input files

In [151]:
# Add Input Data *********************************** 
# Massachusetts 2016 Land Use Land Cover Layer 
path = os.path.join(idr,"lclu_gdb","MA_LCLU2016.gdb","LANDCOVER_LANDUSE_POLY")
MA_LCLU2016 = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_lclu16")

# Plymouth tax parcel data
path = r'C:\\Workspace\\Geodata\\Massachusetts\\L3_SHP_M239_Plymouth\\'
#ap.management.MakeFeatureLayer(path,"M239TaxPar")
l1 = os.path.join(path,"M239TaxPar.shp")
l2 = os.path.join(path,"M239Assess.dbf")
l1 = arcpy.management.CopyFeatures(l1, "M239TaxPar_Assess", '', None, None, None)
parcel = arcpy.management.AddJoin(l1, "LOC_ID", l2, "LOC_ID", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
# Massachusetts cranberry agriculture (source: Jim McLaughlin)
# this needs to be updated
path = os.path.join(idr,'WMAbogsDRAFT2013\\WMAbogsDRAFT2013.shp')
os.path.exists(path)
arcpy.CopyFeatures_management(path,"in_bogs")
bogs = ap.FeatureSet(path)

# Subcatchments of Great, Green, and Bourne Ponds with terminus classifications
path = os.path.join(idr,"Tidmarsh","tidmarsh_pinehills.shp")
os.path.exists(path)
tidmarsh = ap.FeatureSet(path)
ap.management.MakeFeatureLayer(path,"in_tidmarsh")


<Result 'in_tidmarsh'>

## Main Program

### Part 1: Combine spatial layers

`arcpy.analysis.Intersect` (A) land use and (B) subwatershed Layer. 

In [152]:
# Intersect MA land use layer with subwatershed layer
inFeatures = [MA_LCLU2016, tidmarsh]
intersectOutput = "out_tidmarsh_lclu16"
arcpy.analysis.Intersect(inFeatures, intersectOutput, "", "", "INPUT")

<Result 'C:\\Workspace\\Geodata\\Nload\\out_tidmarsh_lclu16.shp'>

In [167]:
# intersect parcels within tidmarsh watershed. 
inFeatures = [parcel, tidmarsh]
parcel_tidmarsh = "out_tidmarsh_parcel"
arcpy.analysis.Intersect(inFeatures, parcel_tidmarsh,"ALL", None, "INPUT")

<Result 'C:\\Workspace\\Geodata\\Nload\\out_tidmarsh_parcel.shp'>

Spatial (A) join cranberry and (B) intersection of subwatershed land use layers. 

In [154]:
# identify cranberry bogs within intersection of 
inFeat = intersectOutput
identFeat = bogs
identOutput = "out_lclu16_tidmarsh_cran_ident"

In [155]:
arcpy.analysis.Identity(inFeat, identFeat, identOutput, "ALL", None, "NO_RELATIONSHIPS")

<Result 'C:\\Workspace\\Geodata\\Nload\\out_lclu16_tidmarsh_cran_ident.shp'>

Calculate area of polygons in hectares and export attribute tables to pandas dataframe

In [156]:
def fn_Calculate_Area_Ha(fc):
    # calculate area in hectares and add column to attribute table
    arcpy.management.CalculateGeometryAttributes(fc,
                                             "Area_Ha AREA_GEODESIC",
                                             '', "HECTARES", None, "SAME_AS_INPUT")

# path to feature class
fc1 = "in_tidmarsh"
fn_Calculate_Area_Ha(fc1)

In [157]:
fc2 = identOutput
fn_Calculate_Area_Ha(fc2)

fc4 = parcel_tidmarsh
fn_Calculate_Area_Ha(fc4)

In [158]:
def fn_featureClass_Attributes_to_DataFrame(fc):
    # list of field names that excludes the shape geometry (including geometry messes this up)
    flds = [fld.name for fld in arcpy.ListFields(fc) if fld.name != arcpy.Describe(fc).shapeFieldName]
    arr = arcpy.da.FeatureClassToNumPyArray(fc,flds)
    df = pd.DataFrame(arr)
    return(df)

in_df1 = fn_featureClass_Attributes_to_DataFrame(fc1).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})

In [159]:
in_df2 = fn_featureClass_Attributes_to_DataFrame(fc2).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})
in_df4 = fn_featureClass_Attributes_to_DataFrame(fc3).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})

In [160]:
# check that area of the inputs and outputs match closely
Total_Area1 = in_df1.Area_Ha.sum()
Total_Area2 = in_df2.Area_Ha.sum()
Total_Area3 = in_df4.Area_Ha.sum()
print(fc1+" Area (Ha):"+str(Total_Area1))
print(fc2+" Area (Ha):"+str(Total_Area2))
print(fc3+" Area (Ha):"+str(Total_Area3))
#print("Difference in Area:"+str(Total_Area1 - Total_Area2)) # difference should be close to 0.0

in_tidmarsh Area (Ha):1316.824930235
out_lclu16_tidmarsh_cran_ident Area (Ha):1327.6348147163876
out_tidmarsh_parcel Area (Ha):1316.8255482910433


In [161]:
# write attribute tables to csv
in_df1.to_csv(os.path.join(odr,fc1+".csv"))
in_df2.to_csv(os.path.join(odr,fc2+".csv"))
in_df4.to_csv(os.path.join(odr,fc4+".csv"))

### Part 1: Reclassify combined layers for Nload Scenarios

Read excel tables with subwatershed land use area and terminus information

In [168]:
# Subwatershed land use area
inname1 = "in_tidmarsh"
inname2 = "out_lclu16_tidmarsh_cran_ident"
inname4 = "out_tidmarsh_parcel"
in_df1 = pd.read_csv(os.path.join(odr,inname1+'.csv')).rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})

# Subwatershed catchment ids and terminus information
in_df2 = pd.read_csv(os.path.join(odr,inname2+'.csv')).rename(columns={'SHED_ID':'SUB',
                                                                       'Terminal_N':'Terminus',
                                                                       'Bog_Type':'Bog_type'})

# Subwatershed catchment ids and parcel data
in_df4 = pd.read_csv(os.path.join(odr,inname4+'.csv')).rename(columns={'SHED_ID':'SUB',
                                                                       'Terminal_N':'Terminus',
                                                                       'Bog_Type':'Bog_type',
                                                                       'M239Ass_30':'Units',
                                                                      'FID_Pine_1':'PH_True'})

In [163]:
display(in_df1)
display(in_df2)
display(in_df4)

,Unnamed: 0,FID,FID_tid_me,ID,GRIDCODE,Areakm2,Aream2Alb,Shape_Leng,UniqID,HUC8,AREA2MI,Impervious,Area_Ha,Shape_Le_1,FID_Pinehi,Shape_Le_2,Shape_Area
0,0,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,572.457415,30068.217648,-1,23337.825184,5.722410e+06
1,1,1,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,744.367515,30068.217648,1,15385.765319,7.440927e+06


,Unnamed: 0,FID,FID_out_ti,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_tidmar,FID_tid_me,ID,GRIDCODE,Areakm2,Aream2Alb,Shape_Le_1,UniqID,HUC8,AREA2MI,Impervious,Area_Ha,Shape_Le_2,FID_Pinehi,Shape_Le_3,Shape_Ar_1,FID_WMAbog,ID_1,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,REGAREA,CERTAREA,CREDITAREA,PERMITAREA,TOTAREA,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,AREA,PERIMETER,PERMIT_NUM,OWNER_FIRS,OWNER_LAST,AREASACRES,Basin,CropStatus
0,0,0,0,13207471,Impervious,2,Right-of-way,55,000,ROW,2016,239,R10C24,76447.199175,3.616570e+05,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,4.386582e-02,30068.217648,-1,23337.825184,5.722410e+06,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.0,,,1899-12-30,,,0.000,0.000,,,,0.000,,
1,1,1,1,13211472,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,1.683113,6.760998e-02,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,6.760998e-06,30068.217648,-1,23337.825184,5.722410e+06,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.0,,,1899-12-30,,,0.000,0.000,,,,0.000,,
2,2,2,2,13225870,Deciduous Forest,9,Agriculture,7,716,FEE,2016,239,R10C24,1017.792855,7.837370e+03,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,7.837370e-01,30068.217648,-1,23337.825184,5.722410e+06,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.0,,,1899-12-30,,,0.000,0.000,,,,0.000,,
3,3,3,3,13225884,Deciduous Forest,9,Residential - single family,11,101,FEE,2016,239,R10C24,29.313708,6.150000e+01,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,4.483554e-03,30068.217648,-1,23337.825184,5.722410e+06,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.0,,,1899-12-30,,,0.000,0.000,,,,0.000,,
4,4,4,4,13226006,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,36.384776,6.850000e+01,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,5.153471e-03,30068.217648,-1,23337.825184,5.722410e+06,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.0,,,1899-12-30,,,0.000,0.000,,,,0.000,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18341,18341,18341,18146,22454431,Grassland,8,Agriculture,7,710C,FEE,2016,239,R11C24,216.952417,4.198539e+02,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,1.302721e-04,30068.217648,-1,23337.825184,5.722410e+06,111,0,42124002,Gilmore Cranberry Co Inc,Beaver Dam Rd.,Plymouth,4,Red House Bogs,8.1,0.0,0.0,0.0,8.1,Jim McLaughlin,BRP_WMA,2007-01-30,,42124002,32639.689,1277.353,,,Gilmore Cranberry Co Inc,8.065,South Coastal,active
18342,18342,18342,18152,22455101,Palustrine Emergent Wetland,15,Residential - single family,11,101,FEE,2016,239,R11C24,140.610173,2.725000e+02,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,2.725000e-02,30068.217648,-1,23337.825184,5.722410e+06,496,0,abandoned,"Tidmarsh Farms, Inc.",Beaver Dam Rd.,Plymouth,4,,0.0,0.0,0.0,0.0,0.0,Jim McLaughlin,BRP_WMA,2011-01-19,conservation easement to NRCS,42123925,750415.127,15397.158,,,"Tidmarsh Farms, Inc.",185.416,South Coastal,abandoned
18343,18343,18343,12592,15219356,Impervious,2,Right-of-way,55,000,ROW,2016,239,R11C24,282100.451099,1.335206e+06,1,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,4.036372e-09,30068.217648,1,15385.765319,7.440927e+06,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.0,,,1899-12-30,,,0.000,0.000,,,,0.000,,
18344,18344,18344,16372,15248261,Developed Open Space,5,Right-of-way,55,000,ROW,2016,239,R11C24,13.360115,2.319685e+00,1,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,4.036372e-09,30068.217648,1,15385.765319,7.440927e+06,-1,,,,,,0,,0.0,0.0,0.0,0.0,0.0,,,1899-12-30,,,0.000,0.000,,,,0.000,,


,Unnamed: 0,FID,FID_M239Ta,M239TaxPar,M239TaxP_1,M239TaxP_2,M239TaxP_3,M239TaxP_4,M239TaxP_5,M239TaxP_6,M239TaxP_7,M239TaxP_8,M239TaxP_9,M239Tax_10,M239Tax_11,M239Assess,M239Asse_1,M239Asse_2,M239Asse_3,M239Asse_4,M239Asse_5,M239Asse_6,M239Asse_7,M239Asse_8,M239Asse_9,M239Ass_10,M239Ass_11,M239Ass_12,M239Ass_13,M239Ass_14,M239Ass_15,M239Ass_16,M239Ass_17,M239Ass_18,M239Ass_19,M239Ass_20,M239Ass_21,M239Ass_22,M239Ass_23,M239Ass_24,M239Ass_25,M239Ass_26,M239Ass_27,M239Ass_28,M239Ass_29,Units,M239Ass_31,M239Ass_32,M239Ass_33,M239Ass_34,M239Ass_35,M239Ass_36,M239Ass_37,FID_tidmar,FID_tid_me,ID,GRIDCODE,Areakm2,Aream2Alb,Shape_Leng,UniqID,HUC8,AREA2MI,Impervious,Area_Ha,Shape_Le_1,FID_Pinehi,Shape_Le_2,Shape_Area
0,0,0,608,182.342941,1826.752200,,F_912458_2792233,TAX,048,ASSESS,,20120503,,N,239,7223,048-000-038Q-000,F_912458_2792233,98100,210000,500,308600,2020,0.46,19950619,100000,101,42 RAY RD,42,RAY RD,,PLYMOUTH,02360,SULLIVAN WAYNE,PO BOX 3036,NEW BERN,NC,28564,,,,,R25,1950,1112,1,1112,Ranch,1,5,A,7745,239,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,0.182683,30068.217648,-1,23337.825184,5.722410e+06
1,1,1,2067,196.085726,2196.668099,074-000-038-009,F_912243_2789003,FEE,074,ASSESS,,20120503,,N,239,13088,074-000-038-009,F_912243_2789003,149800,158200,0,308000,2020,0.54,19000101,0,101,22 CAROLYN DR,22,CAROLYN DR,,PLYMOUTH,02360,SULLIVAN BRIAN F,22 CAROLYN DR,PLYMOUTH,MA,02360,,,,,R25,1969,2184,2,2184,Raised Ranch,1,11,A,14015,239,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,0.219676,30068.217648,-1,23337.825184,5.722410e+06
2,2,2,2323,266.191868,2304.100557,048-000-036-002,F_911826_2791646,FEE,048,ASSESS,,20120503,,N,239,7179,048-000-036-002,F_911826_2791646,122500,151100,0,273600,2020,0.58,20140630,0,101,150 BARTLETT RD,150,BARTLETT RD,,PLYMOUTH,02360,GIFFORD HARRY E,150 BARTLETT RD,PLYMOUTH,MA,02360,,44480,107,,R25,1975,1338,1,1338,Colonial,2,6,A,7699,239,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,0.230421,30068.217648,-1,23337.825184,5.722410e+06
3,3,3,2331,263.554079,3886.781803,074-000-025-003,F_911793_2791051,FEE,074,ASSESS,,20120503,,N,239,12981,074-000-025-003,F_911793_2791051,125300,154800,800,280900,2020,0.99,20171024,240000,101,163 BARTLETT RD,163,BARTLETT RD,,PLYMOUTH,02360,BURKE JAMES R,163 BARTLETT RD,PLYMOUTH,MA,02360,,49086,251,,R25,1930,1568,1,1568,Ranch,1,5,A,13906,239,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,0.388695,30068.217648,-1,23337.825184,5.722410e+06
4,4,4,2387,172.895961,1651.422680,048-000-043C-023,F_912388_2793568,FEE,048,ASSESS,,20120612,,N,239,7297,048-000-043C-023,F_912388_2793568,225100,161500,3500,390100,2020,0.41,19820101,0,101,7 HOOVER ST,7,HOOVER ST,,PLYMOUTH,02360,BRESNAN JOHN D,23 HOOVER ST,PLYMOUTH,MA,02360,,5185,452,,R25,1989,2766,2,2766,Garrison,2,9,A,7819,239,0,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,0.165150,30068.217648,-1,23337.825184,5.722410e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,2067,2067,25976,107.954584,671.893727,078B-000-011-583,F_904347_2792064,FEE,078B,ASSESS,,20200114,,N,239,29024,078B-000-011-583,F_904347_2792064,220700,187500,0,408200,2020,0.17,20191016,675531,101,43 WOODY NOOK,43,WOODY NOOK,,PLYMOUTH,02360,LINNELL HERBERT J JR,43 WOODY NOOK,PLYMOUTH,MA,02360,,51798,243,,RR,2019,2438,1,2438,Contemporary,1,7,A,32373,239,1,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,0.000024,30068.217648,1,15385.765319,7.440927e+06
2068,2068,2068,25634,864.298894,10843.000702,078B-000-000S-240,F_904292_2792267,FEE,078B,ASSESS,,20181212,,N,239,28452,078B-000-000S-240,F_904292_2792267,0,273400,0,273400,2020,2.68,20170630,4500000,130,0 WOODY NOOK,0,WOODY NOOK,,PLYMOUTH,02360,TOLL MA LAND LIMITED PARTNERSH,250 GIBRALTAR RD,HORSHAM,PA,19044,,48614,176,,RR,0,0,0,0,,,0,A,31786,239,1,0,0,0,0.0,0.0,0.0,0,,0.0,0.0,0.000024,30068.217648,1,15385.765319,7.440927e+06
2069,2069,2069,25977,109.844644,682.816621,078B-000-011-584,F_90431

In [145]:
# Subwatershed housing units, driveways, septics etcs. 
in_df3 = pd.read_excel(r'C:\Users\Adrian.Wiegman\OneDrive - USDA\Research\Nload\Inputs\Nload_Watershed_Info.xlsx','tidmarsh_info',comment="#")
display(in_df3)

df2 = in_df2
in_df2['BOG_FID'] = np.where(in_df2['FID_WMAbog']==-1,None,in_df2['FID_WMAbog'])
in_df2['Bog_type'] = np.where(in_df2['FID_WMAbog']==496,'flowthrough',None)
in_df2['SUB'] = 0
df2['SHED_NAME'] = 'Tidmarsh'


df1 = in_df1
df1['SUB'] = 0
df1['SHED_NAME'] = 'Tidmarsh'
df1['Terminus'] = 'w'
df1 = df1[['SUB','SHED_NAME','Terminus','Area_Ha']]
display(df1)
df1.SepticUnits = 0
df1 = df1.rename(columns={'SHED_NAME':'SubW_Name'})
#df1 = df1.merge(in_df3[['SUB','Terminus','SepticUnits']])
display(df1)

,SubW_Name,SUB,Area_Ha,Terminus,CC7_Ha,CC3_Ha,CC2_Ha,People_On_Septic,Imp_NR_Ac,NC_cropland_Ac,Unnamed: 10,Buildings,SepticUnits
0,Tidmarsh,0,1316.82493,w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,SUB,SHED_NAME,Terminus,Area_Ha
0,0,Tidmarsh,w,1316.82493


,SUB,SubW_Name,Terminus,Area_Ha
0,0,Tidmarsh,w,1316.82493


Initialize a dataframe for Land cover (LUC) area by Subcatchment (SUB) with values for use class 2 (lawns), 3 (non cranberry ag), 7 (impervous, non-road). 

In [146]:
nLUCs = 13
SUBs =  list(in_df1['SUB'])*nLUCs
print(len(list(in_df1['SUB'])))
LUCs = [i+1 for i in range(nLUCs)]*len(list(in_df1['SUB']))
#print(LUCs)
df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
df_SUB_LUC
df_SUB_LUC
display(df_SUB_LUC)
df_merged = None
'''
coverclasses = [2,3,7]
for i in coverclasses:
    df_temp = in_df3
    df_temp['LUC'] = i
    df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
    df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'left')
display(df_merged)
'''

i = 2
j = 13
df_temp = in_df3
df_temp['LUC'] = i
df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
df_merged = df_temp[['SUB','LUC','Area_Ha']]
display(df_merged)
df_temp['LUC'] = j
df_temp['Area_Ha'] = -df_temp['Area_Ha'] 
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)

i = 3 # area to be added
j = 4 # area to be subtracted from 
df_temp = in_df3
df_temp['LUC'] = i
df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)
df_temp['LUC'] = j
df_temp['Area_Ha'] = -df_temp['Area_Ha'] 
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)

i = 7
j = 6
df_temp = in_df3
df_temp['LUC'] = i
df_temp['Area_Ha'] = df_temp['CC'+str(i)+'_Ha']
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)
df_temp['LUC'] = j
df_temp['Area_Ha'] = -df_temp['Area_Ha'] 
df_merged = df_merged.merge(df_temp[['SUB','LUC','Area_Ha']],'outer')
display(df_merged)

df_SUB_LUC_merged = pd.merge(df_SUB_LUC,df_merged,'left')
display(df_SUB_LUC_merged)
print(len(df_SUB_LUC_merged['SUB'].unique()))
df_SUB_LUC_merged.to_csv("df_SUB_LUC_merged.csv")
print(df_SUB_LUC_merged.sort_values(['LUC','SUB']))

1


,SUB,LUC
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
5,0,6
6,0,7
7,0,8
8,0,9
9,0,10


,SUB,LUC,Area_Ha
0,0,2,NaN


,SUB,LUC,Area_Ha
0,0,2,NaN
1,0,13,NaN


,SUB,LUC,Area_Ha
0,0,2,NaN
1,0,13,NaN
2,0,3,NaN


,SUB,LUC,Area_Ha
0,0,2,NaN
1,0,13,NaN
2,0,3,NaN
3,0,4,NaN


,SUB,LUC,Area_Ha
0,0,2,NaN
1,0,13,NaN
2,0,3,NaN
3,0,4,NaN
4,0,7,NaN


,SUB,LUC,Area_Ha
0,0,2,NaN
1,0,13,NaN
2,0,3,NaN
3,0,4,NaN
4,0,7,NaN
5,0,6,NaN


,SUB,LUC,Area_Ha
0,0,1,NaN
1,0,2,NaN
2,0,3,NaN
3,0,4,NaN
4,0,5,NaN
5,0,6,NaN
6,0,7,NaN
7,0,8,NaN
8,0,9,NaN
9,0,10,NaN


1
    SUB  LUC  Area_Ha
0     0    1      NaN
1     0    2      NaN
2     0    3      NaN
3     0    4      NaN
4     0    5      NaN
5     0    6      NaN
6     0    7      NaN
7     0    8      NaN
8     0    9      NaN
9     0   10      NaN
10    0   11      NaN
11    0   12      NaN
12    0   13      NaN


When septic unit data are unavailable. We estimate septic units from the following information sources.

Estimate fraction of total population (k_Pop) living in each subwatershed using one of two methods:
1. Impervious Area (USEGENCODE == 2): dividing the total impervious area by the impervious area within each subwatershed. 
2. Residential Use Codes (USEGENCODE == 10,11,12,13): classify polygons as housing units if they fall under a residential use code

It may be worthwile to add some uncertainty over how many housing units 

https://www.mass.gov/files/documents/2016/08/wr/classificationcodebook.pdf

In [175]:
in_df2["ResidentialUnit"] = (in_df2.COVERCODE == 2) & np.isin(in_df2.USEGENCODE,[10,11,12,13])

USE_CODE_DICT = {
# Residences 
101: 1, # '......Single Family'
102: 5, #'......Condominium' between 
103: 20, #'......Mobile Home (includes land used for purpose of a mobile home park)' 
104: 2, #'......Two-Family'
105: 3, #'......Three-Family'
106: 0, #'......Accessory Land with Improvement - garage'
107: 0, #'......(Intentionally left blank)'
108: 0, #'......(Intentionally left blank)
109: 2, #'......Multiple Houses on one parcel (for example, a single and a two-family on one parcel)'
# Apartments
111: 6,#'......Four to Eight Units'
112: 10#'......More than Eight Unit'
}

totalHousingUnits = in_df4['Units'].sum()
print(totalHousingUnits)
print(totalHousingUnits*2.22)

df_HU_SUB = in_df4.groupby('FID_tidmar')['Units'].sum().reset_index(name='Units')
df_HU_SUB['Units_Fraction'] = df_HU_SUB['Units']/totalHousingUnits
display(df_HU_SUB)

totalHousingUnits = in_df2['ResidentialUnit'].sum()
print(totalHousingUnits)
print(totalHousingUnits*2.22)

df_HU_SUB = in_df2.groupby('FID_tidmar')['ResidentialUnit'].sum().reset_index(name='ResUnits')
df_HU_SUB['ResImp_Fraction'] = df_HU_SUB['ResUnits']/totalHousingUnits
display(df_HU_SUB)

1801
3998.2200000000003


,FID_tidmar,Units,Units_Fraction
0,0,618,0.343143
1,1,1183,0.656857


2107
4677.54


,FID_tidmar,ResUnits,ResImp_Fraction
0,0,1237,0.587091
1,1,870,0.412909


In [85]:
totalImpervious = in_df2.loc[in_df2['COVERCODE']==2, 'Area_Ha'].sum()
df_imp_sub = in_df2.loc[in_df2['COVERCODE']==2].groupby('SUB')['Area_Ha'].sum().reset_index(name='Area_Ha')
df_imp_sub['Impervious_Fraction'] = df_imp_sub['Area_Ha']/totalImpervious
print(df_imp_sub)

   SUB     Area_Ha  Impervious_Fraction
0    0  146.459842                  1.0


In [86]:
df1['k_Pop'] = None
df1 = pd.merge(df1,df_imp_sub[['SUB','Impervious_Fraction']])
df1 = pd.merge(df1,df_HU_SUB[['SUB','ResUnits','ResImp_Fraction']])

# how well do these metrics align
#import matplotlib.pyplot as plt
#df1.plot('Housing_Fraction','Impervious_Fraction','scatter')
#plt.show()

df1['k_Pop'] = df1.loc[:,'ResImp_Fraction']
display(df1)

,SUB,SubW_Name,Terminus,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,0,Tidmarsh,w,1316.82493,1.0,1.0,2107,1.0


Reclassify land use using Nload cover codes for default scenario

In [87]:
# GENERATE DICTIONARY OF CONDITIONS FOR RECLASSIFYING LAND USE

# specific active cranberry: where other farming makes up a non zero portion of cultivated and hay/pasture cover types
specific_active_cranberry = '(np.isin(df.CropStatus,"active"))'

# general active cranberry: use where active cranberry makes up over 99% of COVERCODE 6 and 7. 
general_active_cranberry = '((np.isin(df.CropStatus,"active"))|((df.COVERCODE == 21) & (df.USEGENCODE==7))|(np.isin(df.COVERCODE,[6,7])))'

active_cranberry = specific_active_cranberry
abandoned_cranberry = 'np.isin(df["FID"],df[df.CropStatus == "abandoned"]["FID"])'
flowthrough = 'np.isin(df["FID"],df[df.Bog_type == "flowthrough"]["FID"])'
#abandoned_cranberry = '(np.isin(df.CropStatus,"abandoned"))'
#flowthrough = '(np.isin(df.Bog_type,"flowthrough"))'
waterbody = '(np.isin(df.COVERCODE,[22,21]) & np.invert('+active_cranberry+'))'
# condition dictionary
SPECIFIC = {
    # THE FIRST APPEARING CONDITION TAKES PRIORITY IN NP.SELECT
    # key (LUC): value (condition string)
     # 4: "Agriculture, Active Cranberry, Flowthrough"
    4: active_cranberry+' & '+flowthrough,
    # 5: "Agriculture, Active Cranberry, Non Flowthrough"
    5: active_cranberry+' & (np.invert('+flowthrough+'))',
    # 11: "Retired Cranberry, Flowthrough"
    11: abandoned_cranberry+' & '+flowthrough,
    # 12: "Retired Cranberry, Non flowthrough"
    12: abandoned_cranberry+' & np.invert('+flowthrough+')',
    # 1: "Natural Uplands"
    1: '(df.COVERCODE >= 8) & (df.COVERCODE <= 12)',
     # 6: "Impervious, Roads"
    6: '(df.COVERCODE == 2)',
    # 3: "Agriculture, Non Cranberry"
    3: '(np.isin(df.COVERCODE,[6,7])) & (np.invert('+active_cranberry+'))',
    # 8: "Recieving Body (Estuary)"
    8: '(df.SUB == "0") & ((df.COVERCODE == 23)|'+waterbody+')', # note that SUB is a string
    # 9: "Freshwater Ponds and Lakes"
    9: '(df.SUB != "0") &' + waterbody,
    # 10: "Wetlands"
    10: '(df.COVERCODE >= 13) & (df.COVERCODE <= 18)',
    # 2: "Mowed Areas (Lawns, Sports Fields, and Golf Courses)"
    2: 'df.COVERCODE is [False]', # this will be calculated separately
    # 7: "Impervious, Non-Roads"
    7: 'df.COVERCODE is [False]', # this will be calculated separately
    # 13: "Other (Bare land or Shoreline)"
    13: 'df.COVERCODE is [False]'}

GENERAL = {
    # THE FIRST APPEARING CONDITION TAKES PRIORITY IN NP.SELECT
    # key (LUC): value (condition string)
     # 4: "Agriculture, Active Cranberry, Flowthrough"
    4: active_cranberry+' & '+flowthrough,
    # 5: "Agriculture, Active Cranberry, Non Flowthrough"
    5: active_cranberry+' & (np.invert('+flowthrough+'))',
    # 11: "Retired Cranberry, Flowthrough"
    11: abandoned_cranberry+' & '+flowthrough,
    # 12: "Retired Cranberry, Non flowthrough"
    12: abandoned_cranberry+' & np.invert('+flowthrough+')',
    # 1: "Natural Uplands"
    1: 'df.COVERCODE.between(8, 12, inclusive="both")',
     # 6: "Impervious, Roads"
    6: '(df.COVERCODE == 2) & (df.USEGENCODE == 55)',
    ## 3: "Agriculture, Non Cranberry"
    #3: 'df.COVERCODE == [False]',
    # 8: "Recieving Body (Estuary)"
    8: '(df.SUB == "0") & ((df.COVERCODE == 23)|'+waterbody+')', # note that SUB is a string
    # 9: "Freshwater Ponds and Lakes"
    9: '(df.SUB != "0") &' + waterbody,
    # 10: "Wetlands"
    10: 'df.COVERCODE.between(13, 18, inclusive="both")',
    # 2: "Mowed Areas (Lawns, Sports Fields, and Golf Courses)"
    2: 'df.COVERCODE == 5',
    # 7: "Impervious, Non-Roads"
    7: '(df.COVERCODE == 2) & (df.USEGENCODE != 55)',
    # 13: "Other (Bare land or Shoreline)"
    13: 'np.isin(df.COVERCODE,[19,20])'
}

BASELINE = GENERAL    

# checking condtions 
df = pd.DataFrame({"FID":[0,1,2,3],
                   "COVERCODE":[21,21]*2,
                   "SUB":["0","1"]*2,
                   "CropStatus":["active"]*2 + ["abandoned"]*2,
                   "Bog_type":["","flowthrough"]*2,
                   "USEGENCODE":[0,0,0,0],
                   "USE_CODE":[0,0,0,0]})
df = pd.DataFrame({"FID":[4,5,6,7],
                   "COVERCODE":[2,5]*2,
                   "SUB":["0","1"]*2,
                   "CropStatus":[None]*2 + [None]*2,
                   "Bog_type":[None,None]*2,
                   "USEGENCODE":[55,55,0,0],
                   "USE_CODE":[0,0,0,0]})
df = pd.DataFrame({"FID":[8,9,10,11],
                   "COVERCODE":[9,15,19,22],
                   "SUB":["0","1"]*2,
                   "CropStatus":[None]*2 + [None]*2,
                   "Bog_type":[None,None]*2,
                   "USEGENCODE":[55,55,0,0],
                   "USE_CODE":[0,0,0,0]})
df["8"]= eval(BASELINE[8])
df["9"]= eval(BASELINE[9])
df["11"]= eval(BASELINE[11])
df["12"]= eval(BASELINE[12])
df["2"]= eval(BASELINE[2])
df["6"]= eval(BASELINE[6])
df["7"]= eval(BASELINE[7])
df["13"]= eval(BASELINE[13])
df["10"]= eval(BASELINE[10])
df["1"]= eval(BASELINE[1])

condlist = "["+", ".join(list(BASELINE.values()))+"]"
choicelist = list(BASELINE.keys())
df["LUC"] = np.select(eval(condlist),choicelist,13)
print(df)

   FID  COVERCODE SUB CropStatus Bog_type  ...      7     13     10      1  LUC
0    8          9   0       None     None  ...  False  False  False   True    1
1    9         15   1       None     None  ...  False  False   True  False   10
2   10         19   0       None     None  ...  False   True  False  False   13
3   11         22   1       None     None  ...  False  False  False  False    9

[4 rows x 18 columns]


In [88]:
df = in_df2
#df['Bog_type'] = 'flowthrough'
for i in list(BASELINE.keys()): 
    print("True is in "+str(i)+"....",True in eval(BASELINE[i]))

True is in 4.... False
True is in 5.... True
True is in 11.... True
True is in 12.... False
True is in 1.... True
True is in 6.... True
True is in 8.... True
True is in 9.... True
True is in 10.... True
True is in 2.... True
True is in 7.... True
True is in 13.... True


In [68]:
# declare land use class (LUC) reclasssification function
def fn_reclassify_LUC(df,dictionary,default=13):
    condlist = "["+", ".join(list(dictionary.values()))+"]"
    choicelist = list(dictionary.keys())
    #print(condlist)
    #print(choicelist)
    df["LUC"] =  np.select(eval(condlist),choicelist,default)
    return(df)

In [89]:
# execute reclassification subroutine
df2 = fn_reclassify_LUC(df2,BASELINE)
print("Number of LUC == 13:",sum(df2['LUC']==2))
# replace whitespace data with NaN
df2 = df2.replace(r'^\s*$', np.nan, regex=True)
print(df2.columns)
#df2["LUC"] =  np.where(((df2['LUC']==9) & (df2['SUB']==0)),8,df['LUC'])
# sum land area by subcatchment and LUC
def fn_sum_by_sub_and_luc(df_SUB_LUC_Area1,df_SUB_LUC,df_SUB_LUC_Area2):
    print(df_SUB_LUC)
    df2_SUB_LUC_merged = pd.merge(df_SUB_LUC,df_SUB_LUC_Area1,'left')
    print(df_SUB_LUC)
    Area_Ha = df2_SUB_LUC_merged['Area_Ha'].fillna(0) + df_SUB_LUC_Area2['Area_Ha'].fillna(0)
    print(Area_Ha.sum())
    print(len(Area_Ha))
    df_SUB_LUC_AREA = df_SUB_LUC
    df_SUB_LUC_AREA['Area_Ha'] = Area_Ha
    return(df_SUB_LUC_AREA)

df2_SUB_CC_Area = df2.groupby(['SUB','LUC'])['Area_Ha'].sum().reset_index(name='Area_Ha')
print(df2_SUB_CC_Area)
print(df2_SUB_CC_Area.groupby('LUC')['Area_Ha'].sum())
print(df_SUB_LUC)
print(df_SUB_LUC_merged)
df2_i = fn_sum_by_sub_and_luc(df2_SUB_CC_Area,df_SUB_LUC,df_SUB_LUC_merged)
df2_i.to_csv('df2_SUB_CC_Area.csv')
print(df_SUB_LUC)

Number of LUC == 13: 5289
Index(['Unnamed: 0', 'FID', 'FID_out_ti', 'FID_LANDCO', 'COVERNAME',
       'COVERCODE', 'USEGENNAME', 'USEGENCODE', 'USE_CODE', 'POLY_TYPE', 'FY',
       'TOWN_ID', 'TILENAME', 'Shape_Leng', 'Shape_Area', 'FID_tid_me', 'ID',
       'GRIDCODE', 'Areakm2', 'Aream2Alb', 'Shape_Le_1', 'Shape_Ar_1',
       'UniqID', 'HUC8', 'AREA2MI', 'Impervious', 'Area_Ha', 'FID_WMAbog',
       'ID_1', 'WMA_NO', 'OWNER', 'ADDRESS', 'TOWN', 'REGION', 'BOG_NAME',
       'REGAREA', 'CERTAREA', 'CREDITAREA', 'PERMITAREA', 'TOTAREA', 'STAFF',
       'PROGRAM', 'DATE_ENTER', 'COMMENT', 'BIRTHREG', 'AREA', 'PERIMETER',
       'PERMIT_NUM', 'OWNER_FIRS', 'OWNER_LAST', 'AREASACRES', 'Basin',
       'CropStatus', 'BOG_FID', 'Bog_type', 'SUB', 'ResidentialUnit',
       'SHED_NAME', 'LUC'],
      dtype='object')
   SUB  LUC     Area_Ha
0    0    1  745.528270
1    0    2  206.976862
2    0    5    3.264111
3    0    6   37.356929
4    0    7  109.084788
5    0    9   61.456842
6    0   10  

In [90]:
df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
df_SUB_LUC_merged_general = df_SUB_LUC_merged
df_SUB_LUC_merged_general['Area_Ha'] = 0
print(df_SUB_LUC_merged_general)
df2_i = fn_sum_by_sub_and_luc(df2_SUB_CC_Area,df_SUB_LUC,df_SUB_LUC_merged_general).dropna()
print(df2_i)
# Aggregate Data with Pandas .groupby()
df2_i.to_csv(os.path.join(odr,"Tidmarsh_LUC_Area_general"+".csv"))
display(df2_i)
# check that area is within one tenth of a Ha of input
print('Difference in Total Area (Ha):',round(df2['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

    SUB  LUC  Area_Ha
0     0    1        0
1     0    2        0
2     0    3        0
3     0    4        0
4     0    5        0
5     0    6        0
6     0    7        0
7     0    8        0
8     0    9        0
9     0   10        0
10    0   11        0
11    0   12        0
12    0   13        0
    SUB  LUC
0     0    1
1     0    2
2     0    3
3     0    4
4     0    5
5     0    6
6     0    7
7     0    8
8     0    9
9     0   10
10    0   11
11    0   12
12    0   13
    SUB  LUC
0     0    1
1     0    2
2     0    3
3     0    4
4     0    5
5     0    6
6     0    7
7     0    8
8     0    9
9     0   10
10    0   11
11    0   12
12    0   13
1327.6348145953666
13
    SUB  LUC     Area_Ha
0     0    1  745.528270
1     0    2  206.976862
2     0    3    0.000000
3     0    4    0.000000
4     0    5    3.264111
5     0    6   37.356929
6     0    7  109.084788
7     0    8    0.000000
8     0    9   61.456842
9     0   10   68.040977
10    0   11   55.779958
11    

,SUB,LUC,Area_Ha
0,0,1,745.528270
1,0,2,206.976862
2,0,3,0.000000
3,0,4,0.000000
4,0,5,3.264111
5,0,6,37.356929
6,0,7,109.084788
7,0,8,0.000000
8,0,9,61.456842
9,0,10,68.040977


Difference in Total Area (Ha): 0.0


Reclassify terminus and land use based on cranberry management scenarios

In [91]:
display(df2)
display(df1)
# List of subwatersheds with cranberry at terminus
SUB_LIST_1 = [19,26,29]
# List of subwatersheds with flowthrough cranberry at terminus
SUB_LIST_4 = [19,26,29]
# List of cranberry bogs at terminus of subwatersheds
#BOG_FID_LIST_1 = [4,7,11]
BOG_FID_LIST_1 = [167,584,977]
BOG_FID_LIST_1 = [str(i) for i in BOG_FID_LIST_1]
print("total restored area (Ha):",df2[np.isin(df2['BOG_FID'],BOG_FID_LIST_1)]['Area_Ha'].sum())
print(BOG_FID_LIST_1)

,Unnamed: 0,FID,FID_out_ti,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_tid_me,ID,GRIDCODE,Areakm2,Aream2Alb,Shape_Le_1,Shape_Ar_1,UniqID,HUC8,AREA2MI,Impervious,Area_Ha,FID_WMAbog,ID_1,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,REGAREA,CERTAREA,CREDITAREA,PERMITAREA,TOTAREA,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,AREA,PERIMETER,PERMIT_NUM,OWNER_FIRS,OWNER_LAST,AREASACRES,Basin,CropStatus,BOG_FID,Bog_type,SUB,ResidentialUnit,SHED_NAME,LUC
0,0,0,0,13207471,Impervious,2,Right-of-way,55,000,ROW,2016,239,R10C24,76447.199175,3.616570e+05,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.386562e-02,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,6
1,1,1,1,13211472,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,1.683113,6.760998e-02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,6.760998e-06,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,True,Tidmarsh,7
2,2,2,2,13225870,Deciduous Forest,9,Agriculture,7,716,FEE,2016,239,R10C24,1017.792855,7.837370e+03,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,7.837370e-01,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,1
3,3,3,3,13225884,Deciduous Forest,9,Residential - single family,11,101,FEE,2016,239,R10C24,29.313708,6.150000e+01,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.483546e-03,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,1
4,4,4,4,13226006,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,36.384776,6.850000e+01,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,5.153461e-03,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,True,Tidmarsh,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18256,18256,18256,18062,22454431,Grassland,8,Agriculture,7,710C,FEE,2016,239,R11C24,216.952417,4.198539e+02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,1.302721e-04,111,0,42124002,Gilmore Cranberry Co Inc,Beaver Dam Rd.,Plymouth,4,Red House Bogs,8.1,0.0,0.0,0.0,8.1,Jim McLaughlin,BRP_WMA,2007-01-30,NaN,42124002,32639.689,1277.353,NaN,NaN,Gilmore Cranberry Co Inc,8.065,South Coastal,active,111.0,None,0,False,Tidmarsh,5
18257,18257,18257,18068,22455101,Palustrine Emergent Wetland,15,Residential - single family,11,101,FEE,2016,239,R11C24,140.610173,2.725000e+02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,2.725000e-02,496,0,abandoned,"Tidmarsh Farms, Inc.",Beaver Dam Rd.,Plymouth,4,NaN,0.0,0.0,0.0,0.0,0.0,Jim McLaughlin,BRP_WMA,2011-01-19,conservation easement to NRCS,42123925,750415.127,15397.158,NaN,NaN,"Tidmarsh Farms, Inc.",185.416,South Coastal,abandoned,496.0,flowthrough,0,False,Tidmarsh,11
18258,18258,18258,12544,15219356,Impervious,2,Right-of-way,55,000,ROW,2016,239,R11C24,282100.451099,1.335206e+06,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.036372e-09,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,6
18259,18259,18259,16297,15248261,Developed Open Space,5,Right-of-way,55,000,ROW,2016,239,R11C24,13.360115,2.319685e+00,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.036372e-09,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,2


,SUB,SubW_Name,Terminus,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,0,Tidmarsh,w,1316.82493,1.0,1.0,2107,1.0


C:\Users\Adrian.Wiegman\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


total restored area (Ha): 0.0
['167', '584', '977']


In [100]:
display(df2)
# Dictionary of Scenario Conditions
LUC_scenarios = {
    # 0: "Baseline (Do nothing)"
    0: 'False',
    #1: "Restore Select Bogs to Wetlands"
    1: "np.isin(df2['BOG_FID'],BOG_FID_LIST_1)",
    #2: "Restore All Retired Bogs to Wetlands"
    2: "np.isin(df2['FID'],df2[df2.CropStatus == 'abandoned']['FID'])",
    #3: "Retire All Flowthrough Bogs"
    3: "np.isin(df2['FID'],df2[df2.Bog_type == 'flowthrough']['FID'])",
    #4: "Restore All Flowthrough Bogs to Wetlands"
    4: "np.isin(df2['FID'],df2[df2.Bog_type == 'flowthrough']['FID'])"
}
Terminus_scenarios = {
    # 0: "Baseline (Do nothing)"
    0: '[False]',
    #1: "Restore Select Bogs to Wetlands"
    1: 'np.isin(df1[\'SUB\'],SUB_LIST_1)',
    #2: "Restore All Retired Bogs to Wetlands"
    2: '[False]',
    #3: "Retire All Flowthrough Bogs"
    3: '[False]',
    #4: "Restore All Flowthrough Bogs to Wetlands"
    4: 'np.isin(df1[\'SUB\'],SUB_LIST_4)'
}
#df2 = df2.replace(r'^\s*$', np.nan, regex=True)
#display(df2)
#True in [i is 'abandoned' for i in df2.CropStatus]
#Areasum = df2[df2.CropStatus == 'abandoned'].groupby(['SubW_Name','LUC'])['Area_Ha'].sum()
Areasum = df2[df2.CropStatus == 'abandoned'].groupby(['SHED_NAME','LUC'])['Area_Ha'].sum()
print(Areasum)
print(df2[df2.FID==59495]['CropStatus'] == 'abandoned')
print(df2[df2.FID==114647]['Bog_type'] == 'flowthrough')
#print([i is "abandoned" for i in df2.CropStatus.astype("|S")])
print(df2[['BOG_FID','CropStatus','SUB']].dtypes)
print(eval(LUC_scenarios[2]))
df2.BOG_FID.to_csv("BOG_FID.csv")
print("Is 'True' in the result: ",True in eval(Terminus_scenarios[4]))
Terminus_scenarios.keys()
Terminus_scenarios.values()

,Unnamed: 0,FID,FID_out_ti,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_tid_me,ID,GRIDCODE,Areakm2,Aream2Alb,Shape_Le_1,Shape_Ar_1,UniqID,HUC8,AREA2MI,Impervious,Area_Ha,FID_WMAbog,ID_1,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,REGAREA,CERTAREA,CREDITAREA,PERMITAREA,TOTAREA,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,AREA,PERIMETER,PERMIT_NUM,OWNER_FIRS,OWNER_LAST,AREASACRES,Basin,CropStatus,BOG_FID,Bog_type,SUB,ResidentialUnit,SHED_NAME,LUC,LUC0,LUC1,LUC2,LUC3,LUC4
0,0,0,0,13207471,Impervious,2,Right-of-way,55,000,ROW,2016,239,R10C24,76447.199175,3.616570e+05,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.386562e-02,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,6,6,6,6,6,6
1,1,1,1,13211472,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,1.683113,6.760998e-02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,6.760998e-06,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,True,Tidmarsh,7,7,7,7,7,7
2,2,2,2,13225870,Deciduous Forest,9,Agriculture,7,716,FEE,2016,239,R10C24,1017.792855,7.837370e+03,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,7.837370e-01,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,1,1,1,1,1,1
3,3,3,3,13225884,Deciduous Forest,9,Residential - single family,11,101,FEE,2016,239,R10C24,29.313708,6.150000e+01,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.483546e-03,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,1,1,1,1,1,1
4,4,4,4,13226006,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,36.384776,6.850000e+01,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,5.153461e-03,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,True,Tidmarsh,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18256,18256,18256,18062,22454431,Grassland,8,Agriculture,7,710C,FEE,2016,239,R11C24,216.952417,4.198539e+02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,1.302721e-04,111,0,42124002,Gilmore Cranberry Co Inc,Beaver Dam Rd.,Plymouth,4,Red House Bogs,8.1,0.0,0.0,0.0,8.1,Jim McLaughlin,BRP_WMA,2007-01-30,NaN,42124002,32639.689,1277.353,NaN,NaN,Gilmore Cranberry Co Inc,8.065,South Coastal,active,111.0,None,0,False,Tidmarsh,5,5,5,5,5,5
18257,18257,18257,18068,22455101,Palustrine Emergent Wetland,15,Residential - single family,11,101,FEE,2016,239,R11C24,140.610173,2.725000e+02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,2.725000e-02,496,0,abandoned,"Tidmarsh Farms, Inc.",Beaver Dam Rd.,Plymouth,4,NaN,0.0,0.0,0.0,0.0,0.0,Jim McLaughlin,BRP_WMA,2011-01-19,conservation easement to NRCS,42123925,750415.127,15397.158,NaN,NaN,"Tidmarsh Farms, Inc.",185.416,South Coastal,abandoned,496.0,flowthrough,0,False,Tidmarsh,11,11,11,10,11,10
18258,18258,18258,12544,15219356,Impervious,2,Right-of-way,55,000,ROW,2016,239,R11C24,282100.451099,1.335206e+06,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.036372e-09,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,6,6,6,6,6,6
18259,18259,18259,16297,15248261,Developed Open Space,5,Right-of-way,55,000,ROW,2016,239,R11C24,13.360115,2.319685e+00,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.036372e-09,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,2,2,2,2,2,2


SHED_NAME  LUC
Tidmarsh   11     55.779958
Name: Area_Ha, dtype: float64
Series([], Name: CropStatus, dtype: bool)
Series([], Name: Bog_type, dtype: bool)
BOG_FID       float64
CropStatus     object
SUB             int64
dtype: object
[False False False ... False False False]
Is 'True' in the result:  False


dict_values(['[False]', "np.isin(df1['SUB'],SUB_LIST_1)", '[False]', '[False]', "np.isin(df1['SUB'],SUB_LIST_4)"])

In [101]:
# For each scenario reclassify land use, then sum up area by land use and subwatershed
# default value if scenario condition is true
LUC_iftrue = 10
Terminus_iftrue = "w"
scens = [str(i) for i in range(5)]
list(LUC_scenarios.keys())

[0, 1, 2, 3, 4]

In [102]:
# For each scenario reclassify land use,
out_df2 = df2
out_df1 = df1
# switch Terminus of Maple Swamp from w to s
#rownums = np.where(out_df1['SubW_Name']=='Maple Swamp')[0]
#out_df1.at[rownums,'Terminus']='s'
# default LUC value if scenario condition is true
LUC_iftrue = 10
# default Terminus value
Terminus_iftrue = "w"
for i in list(LUC_scenarios.keys()):
    print (i)
    # alternative land use for scenario 
    if i == 3: 
        LUC_iftrue = 11
    else: 
        LUC_iftrue = 10
        
    # alter LUC based on scenario conditions
    #np.where(logic,if_true, if_false)
    out_df2["LUC"+str(i)] = np.where(eval(LUC_scenarios[i]),LUC_iftrue,df2.LUC)
    sum((np.where(eval(LUC_scenarios[i]),LUC_iftrue,df2.LUC)))
    out_df1["Terminus"+str(i)] = np.where(eval(Terminus_scenarios[i]),Terminus_iftrue,df1.Terminus)
display(out_df2)
display(out_df1)

print(out_df2.groupby('LUC0')['Area_Ha'].sum())
print(out_df2.groupby('LUC1')['Area_Ha'].sum())
print(out_df2.groupby('LUC2')['Area_Ha'].sum())
print(out_df2.groupby('LUC3')['Area_Ha'].sum())
print(out_df2.groupby('LUC4')['Area_Ha'].sum())

#Save the scenario reclassifications to csv
out_path1 = os.path.join(odr,inname1+'_reclass.csv')
out_path2 = os.path.join(odr,inname2+'_reclass.csv')
out_df1.to_csv(out_path1,index=False)
out_df2.to_csv(out_path2,index=False)

0
1


C:\Users\Adrian.Wiegman\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


2
3
4


,Unnamed: 0,FID,FID_out_ti,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,Shape_Area,FID_tid_me,ID,GRIDCODE,Areakm2,Aream2Alb,Shape_Le_1,Shape_Ar_1,UniqID,HUC8,AREA2MI,Impervious,Area_Ha,FID_WMAbog,ID_1,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,REGAREA,CERTAREA,CREDITAREA,PERMITAREA,TOTAREA,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,AREA,PERIMETER,PERMIT_NUM,OWNER_FIRS,OWNER_LAST,AREASACRES,Basin,CropStatus,BOG_FID,Bog_type,SUB,ResidentialUnit,SHED_NAME,LUC,LUC0,LUC1,LUC2,LUC3,LUC4
0,0,0,0,13207471,Impervious,2,Right-of-way,55,000,ROW,2016,239,R10C24,76447.199175,3.616570e+05,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.386562e-02,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,6,6,6,6,6,6
1,1,1,1,13211472,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,1.683113,6.760998e-02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,6.760998e-06,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,True,Tidmarsh,7,7,7,7,7,7
2,2,2,2,13225870,Deciduous Forest,9,Agriculture,7,716,FEE,2016,239,R10C24,1017.792855,7.837370e+03,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,7.837370e-01,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,1,1,1,1,1,1
3,3,3,3,13225884,Deciduous Forest,9,Residential - single family,11,101,FEE,2016,239,R10C24,29.313708,6.150000e+01,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.483546e-03,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,1,1,1,1,1,1
4,4,4,4,13226006,Impervious,2,Residential - single family,11,101,FEE,2016,239,R10C24,36.384776,6.850000e+01,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,5.153461e-03,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,True,Tidmarsh,7,7,7,7,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18256,18256,18256,18062,22454431,Grassland,8,Agriculture,7,710C,FEE,2016,239,R11C24,216.952417,4.198539e+02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,1.302721e-04,111,0,42124002,Gilmore Cranberry Co Inc,Beaver Dam Rd.,Plymouth,4,Red House Bogs,8.1,0.0,0.0,0.0,8.1,Jim McLaughlin,BRP_WMA,2007-01-30,NaN,42124002,32639.689,1277.353,NaN,NaN,Gilmore Cranberry Co Inc,8.065,South Coastal,active,111.0,None,0,False,Tidmarsh,5,5,5,5,5,5
18257,18257,18257,18068,22455101,Palustrine Emergent Wetland,15,Residential - single family,11,101,FEE,2016,239,R11C24,140.610173,2.725000e+02,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,2.725000e-02,496,0,abandoned,"Tidmarsh Farms, Inc.",Beaver Dam Rd.,Plymouth,4,NaN,0.0,0.0,0.0,0.0,0.0,Jim McLaughlin,BRP_WMA,2011-01-19,conservation easement to NRCS,42123925,750415.127,15397.158,NaN,NaN,"Tidmarsh Farms, Inc.",185.416,South Coastal,abandoned,496.0,flowthrough,0,False,Tidmarsh,11,11,11,10,11,10
18258,18258,18258,12544,15219356,Impervious,2,Right-of-way,55,000,ROW,2016,239,R11C24,282100.451099,1.335206e+06,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.036372e-09,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,6,6,6,6,6,6
18259,18259,18259,16297,15248261,Developed Open Space,5,Right-of-way,55,000,ROW,2016,239,R11C24,13.360115,2.319685e+00,0,0,0,0.0,0.0,0.0,0.0,0,NaN,0.0,0.0,4.036372e-09,-1,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1899-12-30,NaN,NaN,0.000,0.000,NaN,NaN,NaN,0.000,NaN,NaN,NaN,None,0,False,Tidmarsh,2,2,2,2,2,2


,SUB,SubW_Name,Terminus,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,Terminus0,Terminus1,Terminus2,Terminus3,Terminus4
0,0,Tidmarsh,w,1316.82493,1.0,1.0,2107,1.0,w,w,w,w,w


LUC0
1     745.528270
2     206.976862
5       3.264111
6      37.356929
7     109.084788
9      61.456842
10     68.040977
11     55.779958
13     40.146078
Name: Area_Ha, dtype: float64
LUC1
1     745.528270
2     206.976862
5       3.264111
6      37.356929
7     109.084788
9      61.456842
10     68.040977
11     55.779958
13     40.146078
Name: Area_Ha, dtype: float64
LUC2
1     745.528270
2     206.976862
5       3.264111
6      37.356929
7     109.084788
9      61.456842
10    123.820935
13     40.146078
Name: Area_Ha, dtype: float64
LUC3
1     745.528270
2     206.976862
5       3.264111
6      37.356929
7     109.084788
9      61.456842
10     68.040977
11     55.779958
13     40.146078
Name: Area_Ha, dtype: float64
LUC4
1     745.528270
2     206.976862
5       3.264111
6      37.356929
7     109.084788
9      61.456842
10    123.820935
13     40.146078
Name: Area_Ha, dtype: float64


In [103]:
#copy the original layer and add to the map
#fc1_reclass = arcpy.management.CopyFeatures(fc1, fc1+'_reclass')
fc2_reclass = arcpy.management.CopyFeatures(inname2, inname2+'_reclass')
print(fc2_reclass)
# Join the scenarios csv files with the copy of original layers
#arcpy.management.AddJoin(fc1_reclass, "FID", out_path1, "FID", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
fc2_reclass = arcpy.management.AddJoin(fc2_reclass, "FID", out_path2, "FID", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")



C:\Workspace\Geodata\Nload\out_lclu16_tidmarsh_cran_ident_reclass.shp


In [104]:
# Conditions for terminus transmission coefficient
df1_dict = {'w': 0.23,
'c': 1,
'e': 1,
's': 1,
'p': 1}
df1_conditions = '['+', '.join(['df1_i.Terminus =="' + k+'"' for k in df1_dict.keys()])+']'
df2_dict = {'9': 1,
'10': 1}
df2_conditions = '['+', '.join(['df2_i.Terminus =="' + k+'"' for k in df2_dict.keys()])+']'
# change terminus classifications    
for i in (LUC_scenarios.keys()):
    print (i) 

    df1_i = pd.merge(out_df1[["SUB","Terminus"+str(i)]],
                     out_df1.loc[:, np.invert(out_df1.columns.str.startswith('Terminus'))])
    df1_i = df1_i.rename(columns={'Terminus'+str(i):'Terminus'})
    df1_i['c_T'] = np.select(eval(df1_conditions),df1_dict.values(),default=1)
    display(df1_i)
    df1_i.to_csv(os.path.join(odr,"GGBP_Terminus_and_Pop"+str(i)+".csv"))

    # then sum up area by land use and subwatershed and assign terminus coefficients
    df2_i = out_df2[['SUB','LUC'+str(i),'Area_Ha']]
    df2_i = df2_i.groupby(['SUB','LUC'+str(i)])['Area_Ha'].sum().reset_index(name='Area_Ha')
    df2_i = df2_i.rename(columns={'LUC'+str(i):'LUC'})
    """    
    df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
    display(df_SUB_LUC)
    display(df_SUB_LUC_merged)
    df2_i = pd.merge(df_SUB_LUC,df2_i,'left')
    Area_Ha = df_SUB_LUC_merged['Area_Ha'].fillna(0) + df2_i['Area_Ha'].fillna(0)
    print(Area_Ha.sum())
    print(len(Area_Ha))
    df_SUB_LUC_AREA = df_SUB_LUC
    df_SUB_LUC_AREA['Area_Ha'] = Area_Ha
    df2_i = pd.merge(df_SUB_LUC_AREA,df1_i[['SUB','Terminus']])
    df2_i['c_T'] = np.select(eval(df2_conditions),df2_dict.values(),default=1)
    print("!!!!!!!!!!!!displaying df2_i after merge")
    """
    display(df2_i)
    df2_i.to_csv(os.path.join(odr,"tidmarsh_LUC_Area_p1_"+str(i)+".csv"))
    
    # check that area is within one tenth of a Ha of input
    print('Difference in Total Area (Ha):',round(out_df1['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

0


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0,0.23


,SUB,LUC,Area_Ha
0,0,1,745.528270
1,0,2,206.976862
2,0,5,3.264111
3,0,6,37.356929
4,0,7,109.084788
5,0,9,61.456842
6,0,10,68.040977
7,0,11,55.779958
8,0,13,40.146078


Difference in Total Area (Ha): -10.8
1


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0,0.23


...

,SUB,LUC,Area_Ha
0,0,1,745.528270
1,0,2,206.976862
2,0,5,3.264111
3,0,6,37.356929
4,0,7,109.084788
5,0,9,61.456842
6,0,10,68.040977
7,0,11,55.779958
8,0,13,40.146078


Difference in Total Area (Ha): -10.8
4


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0,0.23


,SUB,LUC,Area_Ha
0,0,1,745.528270
1,0,2,206.976862
2,0,5,3.264111
3,0,6,37.356929
4,0,7,109.084788
5,0,9,61.456842
6,0,10,123.820935
7,0,13,40.146078


Difference in Total Area (Ha): -10.8


In [105]:
df1_dict = {'w': 0.23,
'c': 1,
'e': 1,
's': 1,
'p': 0.5}
df1_conditions = '['+', '.join(['df1_i.Terminus =="' + k+'"' for k in df1_dict.keys()])+']'
df2_dict = {'9': 1,
'10': 1}
df2_conditions = '['+', '.join(['df2_i.Terminus =="' + k+'"' for k in df2_dict.keys()])+']'
# change terminus classifications    
for i in (LUC_scenarios.keys()):
    print (i) 

    df1_i = pd.merge(out_df1[["SUB","Terminus"+str(i)]],
                     out_df1.loc[:, np.invert(out_df1.columns.str.startswith('Terminus'))])
    df1_i = df1_i.rename(columns={'Terminus'+str(i):'Terminus'})
    df1_i['c_T'] = np.select(eval(df1_conditions),df1_dict.values(),default=1)
    display(df1_i)
    df1_i.to_csv(os.path.join(odr,"GGBP_Terminus_and_Pop"+str(i)+".csv"))

    # then sum up area by land use and subwatershed and assign terminus coefficients
    df2_i = out_df2[['SUB','LUC'+str(i),'Area_Ha']]
    df2_i = df2_i.groupby(['SUB','LUC'+str(i)])['Area_Ha'].sum().reset_index(name='Area_Ha')
    df2_i = df2_i.rename(columns={'LUC'+str(i):'LUC'})
    '''
    df_SUB_LUC = pd.DataFrame({"SUB":SUBs,"LUC":LUCs})
    display(df_SUB_LUC)
    display(df_SUB_LUC_merged)
    df2_i = pd.merge(df_SUB_LUC,df2_i,'left')
    Area_Ha = df_SUB_LUC_merged['Area_Ha'].fillna(0) + df2_i['Area_Ha'].fillna(0)
    print(Area_Ha.sum())
    print(len(Area_Ha))
    df_SUB_LUC_AREA = df_SUB_LUC
    df_SUB_LUC_AREA['Area_Ha'] = Area_Ha
    df2_i = pd.merge(df_SUB_LUC_AREA,df1_i[['SUB','Terminus']])
    df2_i['c_T'] = np.select(eval(df2_conditions),df2_dict.values(),default=1)
    print("!!!!!!!!!!!!displaying df2_i after merge")
    '''
    display(df2_i)
    df2_i.to_csv(os.path.join(odr,"tidmarsh_LUC_Area_p05_"+str(i)+".csv"))
    
    # check that area is within one tenth of a Ha of input
    print('Difference in Total Area (Ha):',round(out_df1['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

0


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0,0.23


,SUB,LUC,Area_Ha
0,0,1,745.528270
1,0,2,206.976862
2,0,5,3.264111
3,0,6,37.356929
4,0,7,109.084788
5,0,9,61.456842
6,0,10,68.040977
7,0,11,55.779958
8,0,13,40.146078


Difference in Total Area (Ha): -10.8
1


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0,0.23


...

,SUB,LUC,Area_Ha
0,0,1,745.528270
1,0,2,206.976862
2,0,5,3.264111
3,0,6,37.356929
4,0,7,109.084788
5,0,9,61.456842
6,0,10,68.040977
7,0,11,55.779958
8,0,13,40.146078


Difference in Total Area (Ha): -10.8
4


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction,c_T
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0,0.23


,SUB,LUC,Area_Ha
0,0,1,745.528270
1,0,2,206.976862
2,0,5,3.264111
3,0,6,37.356929
4,0,7,109.084788
5,0,9,61.456842
6,0,10,123.820935
7,0,13,40.146078


Difference in Total Area (Ha): -10.8


In [108]:
# GENERAL 
# change terminus classifications    
for i in (LUC_scenarios.keys()):
    print (i) 

    df1_i = pd.merge(out_df1[["SUB","Terminus"+str(i)]],
                     out_df1.loc[:, np.invert(out_df1.columns.str.startswith('Terminus'))])
    df1_i = df1_i.rename(columns={'Terminus'+str(i):'Terminus'})
    #df1_i['c_T'] = np.select(eval(df1_conditions),df1_dict.values(),default=1)
    display(df1_i)
    df1_i.to_csv(os.path.join(odr,"tidmarsh_Terminus_and_Pop_"+str(i)+".csv"))

    # then sum up area by land use and subwatershed and assign terminus coefficients
    df2_i = out_df2[['SUB','LUC'+str(i),'Area_Ha']]
    df2_i = df2_i.groupby(['SUB','LUC'+str(i)])['Area_Ha'].sum().reset_index(name='Area_Ha')
    df2_i = df2_i.rename(columns={'LUC'+str(i):'LUC'})
    df2_i = pd.merge(df2_i,df1_i[['SUB','Terminus']],'inner')
    display(df2_i)
    df2_i.to_csv(os.path.join(odr,"tidmarsh_LUC_Area_"+str(i)+".csv"))
    
    # check that area is within one tenth of a Ha of input
    print('Difference in Total Area (Ha):',round(out_df1['Area_Ha'].sum() - df2_i['Area_Ha'].sum(),1))

0


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0


,SUB,LUC,Area_Ha,Terminus
0,0,1,745.528270,w
1,0,2,206.976862,w
2,0,5,3.264111,w
3,0,6,37.356929,w
4,0,7,109.084788,w
5,0,9,61.456842,w
6,0,10,68.040977,w
7,0,11,55.779958,w
8,0,13,40.146078,w


Difference in Total Area (Ha): -10.8
1


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0


...

,SUB,LUC,Area_Ha,Terminus
0,0,1,745.528270,w
1,0,2,206.976862,w
2,0,5,3.264111,w
3,0,6,37.356929,w
4,0,7,109.084788,w
5,0,9,61.456842,w
6,0,10,68.040977,w
7,0,11,55.779958,w
8,0,13,40.146078,w


Difference in Total Area (Ha): -10.8
4


,SUB,Terminus,SubW_Name,Area_Ha,k_Pop,Impervious_Fraction,ResUnits,ResImp_Fraction
0,0,w,Tidmarsh,1316.82493,1.0,1.0,2107,1.0


,SUB,LUC,Area_Ha,Terminus
0,0,1,745.528270,w
1,0,2,206.976862,w
2,0,5,3.264111,w
3,0,6,37.356929,w
4,0,7,109.084788,w
5,0,9,61.456842,w
6,0,10,123.820935,w
7,0,13,40.146078,w


Difference in Total Area (Ha): -10.8


In [203]:
print(fc2)
fc2df = fn_featureClass_Attributes_to_DataFrame(fc2)#.rename(columns={'SHED_ID':'SUB','Terminal_N':'Terminus'})
in_cols = fc2df.columns.to_list()
print(in_cols)
out_cols = out_df2.columns.to_list()
condArray = [oc not in in_cols for oc in out_cols]
print(condArray)
print(out_cols)
x = np.compress(condArray,out_cols)
print(*x)
select_cols = [*x, 'FID']
print(select_cols)
#select_cols = filter(condition,out_cols)

out_df2_select = out_df2[select_cols]
out_path_select = os.path.join(odr,identOutput+"_reclass_select.csv")
out_df2_select.to_csv(out_path_select)

out_lclu16_tidmarsh_cran_ident
['FID', 'FID_out_ti', 'FID_LANDCO', 'COVERNAME', 'COVERCODE', 'USEGENNAME', 'USEGENCODE', 'USE_CODE', 'POLY_TYPE', 'FY', 'TOWN_ID', 'TILENAME', 'Shape_Leng', 'Shape_Area', 'FID_tid_me', 'ID', 'GRIDCODE', 'Areakm2', 'Aream2Alb', 'Shape_Le_1', 'Shape_Ar_1', 'UniqID', 'HUC8', 'AREA2MI', 'Impervious', 'Area_Ha', 'FID_WMAbog', 'ID_1', 'WMA_NO', 'OWNER', 'ADDRESS', 'TOWN', 'REGION', 'BOG_NAME', 'REGAREA', 'CERTAREA', 'CREDITAREA', 'PERMITAREA', 'TOTAREA', 'STAFF', 'PROGRAM', 'DATE_ENTER', 'COMMENT', 'BIRTHREG', 'AREA', 'PERIMETER', 'PERMIT_NUM', 'OWNER_FIRS', 'OWNER_LAST', 'AREASACRES', 'Basin', 'CropStatus']
[True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, F

In [212]:
print(pd.read_csv(out_path_select))
# Set local variables
inFeature = fc2
joinTable = out_path_select
joinField = "FID"
tempFeatureName = "temp"
outFeatureName = inFeature+"_reclass_output"

       Unnamed: 0  Unnamed: 0.1  BOG_FID     Bog_type  ...  LUC2  LUC3 LUC4    FID
0               0             0      NaN          NaN  ...     6     6    6      0
1               1             1      NaN          NaN  ...     7     7    7      1
2               2             2      NaN          NaN  ...     1     1    1      2
3               3             3      NaN          NaN  ...     1     1    1      3
4               4             4      NaN          NaN  ...     7     7    7      4
...           ...           ...      ...          ...  ...   ...   ...  ...    ...
18256       18256         18256    111.0          NaN  ...     5     5    5  18256
18257       18257         18257    496.0  flowthrough  ...    10    11   10  18257
18258       18258         18258      NaN          NaN  ...     6     6    6  18258
18259       18259         18259      NaN          NaN  ...     2     2    2  18259
18260       18260         18260      NaN          NaN  ...     2     2    2  18260

[18

In [214]:
# Copy the layer to a new permanent feature class
tempFeature = arcpy.management.CopyFeatures(inFeature,tempFeatureName)

In [219]:
# Join the feature layer to a table
joined_table = arcpy.management.AddJoin(tempFeature, joinField, joinTable,joinField)
#output_feat = joined_table.getOutput(0)
#output_feat.save(outFeatureName)
arcpy.management.CopyFeatures(joined_table,outFeatureName)

arcpy.conversion.PolygonToRaster(joined_table, "out_lclu16_tidmarsh_cran_ident_reclass_select.csv.LUC", outFeatureName+"_PolygonToRaster", "CELL_CENTER", "NONE", 5, "BUILD")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000445: Extension is invalid for the output raster format.
Failed to execute (PolygonToRaster).






















## Appendix

Generate a table of area for 13 land use classes within each subcatchment, by sorting land uses from the following categories. 

Class Number|	Class Name
---|---
2|	Impervious
5|	Developed Open Space
6|	Cultivated Land
7|	Pasture/Hay
8|	Grassland
9|	Deciduous Forest
10|	Evergreen Forest
12|	Scrub/Shrub
13|	Palustrine Forested Wetland (C-CAP)
14|	Palustrine Scrub/Shrub Wetland (C-CAP)
15|	Palustrine Emergent Wetland (C-CAP)
16|	Estuarine Forested Wetland (C-CAP)
17|	Estuarine Scrub/Shrub Wetland (C-CAP)
18|	Estuarine Emergent Wetland (C-CAP)
19|	Unconsolidated Shore
20|	Bare Land
21|	Open Water
22|	Palustrine Aquatic Bed (C-CAP)
23|	Estuarine Aquatic Bed (C-CAP)


USEGENCODE|	USEGENNAME
---|---
0|	Unknown
2|	Open land
3|	Commercial
4|	Industrial
6|	Forest
7|	Agriculture
8|	Recreation
9|	Tax exempt
10| Mixed use, primarily residential
11|	Residential - single family
12|	Residential - multi-family
13|	Residential - other
20|	Mixed use, other
30|	Mixed use, primarily commercial
55|	Right-of-way
88|	Water